In [1]:
from flask import Flask, request, jsonify
import os
import torch
import torch_npu
from torch_npu.contrib import transfer_to_npu
from transformers import AutoTokenizer, AutoModel
import gc
app = Flask(__name__)

tokenizer = AutoTokenizer.from_pretrained("/home/ma-user/work/ctglm/chatglm-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("/home/ma-user/work/ctglm/chatglm-6b", trust_remote_code=True).half().cuda()
##

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/torch_npu/contrib/transfer_to_npu.py:301: ImportWarning: 
    *************************************************************************************************************
    The torch.Tensor.cuda and torch.nn.Module.cuda are replaced with torch.Tensor.npu and torch.nn.Module.npu now..
    The torch.cuda.DoubleTensor is replaced with torch.npu.FloatTensor cause the double type is not supported now..
    The backend in torch.distributed.init_process_group set to hccl now..
    The torch.cuda.* and torch.cuda.amp.* are replaced with torch.npu.* and torch.npu.amp.* now..
    The device parameters have been replaced with npu in the function below:
    torch.logspace, torch.randint, torch.hann_window, torch.rand, torch.full_like, torch.ones_like, torch.rand_like, torch.randperm, torch.arange, torch.frombuffer, torch.normal, torch._empty_per_channel_affine_quantized, torch.empty_strided, torch.empty_like, torch.scalar_tensor

In [ ]:
history = []
MAX_HISTORY_LENGTH = 3
@app.route('/posttext', methods=['POST'])
def post_text():
    global history
    # 检查请求中是否有数据
    if request.is_json:
        # 获取 JSON 请求体中的数据
        data = request.get_json()
        # 假设我们期待的是一个包含 'text' 键的 JSON 对象
        text = data.get('text', '')
        # 处理文本或者直接返回
        response, history = model.chat(tokenizer, text, history=history)
        history.append((text, response))
        if len(history) > MAX_HISTORY_LENGTH:
            history = history[-MAX_HISTORY_LENGTH:]
        # print(response)
        return jsonify({"received_text": response})
    else:
        # 如果不是 JSON 数据，尝试直接从表单数据中获取文本
        text = request.form.get('text', '')
        return jsonify({"received_text": text})
    
    
@app.route('/clearhistory', methods=['POST'])
def clear_history():
    global history
    history = []
    gc.collect()
    torch.npu.empty_cache()
    print("历史记录已清除，并尝试释放内存。")
    
    
if __name__ == '__main__':
    app.run(debug=False, use_reloader=False,port=5004)
    
    
# # 发送表单数据
# curl -X POST http://localhost:5000/posttext -d "text=这是一段文本"

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5004
Press CTRL+C to quit


In [ ]:



# def clear_history():
#     global history
#     history = []
#     gc.collect()
#     torch.npu.empty_cache()
#     print("历史记录已清除，并尝试释放内存。")


# #clear_history()